# Example: LFPy test suite

This example notebook submits a job to a chosen compute cluster (**JUSUF**), loads a **Singularity** container with a working **LFPy** installation and runs **LFPy**'s built in unit test suite (`LFPy.run_tests()`) before exiting. 

In [1]:
# use the pyunicore library
!pip install pyunicore --upgrade

Requirement already up-to-date: pyunicore in /opt/app-root/lib/python3.6/site-packages (0.8.0)
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [23]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint

In [6]:
unicore_client._HBP_REGISTRY_URL

'https://hbp-unic.fz-juelich.de:7112/HBP/rest/registries/default_registry'

In [7]:
# Create connection to supercomputer (i.e., JUSUF)
supercomputer = 'JUSUF'
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [9]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

{'RWTH': 'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core',
 'FZJ_JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core',
 'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'irene': 'https://hbp-unicore.ccc.cea.fr/irene/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-GALILEO': 'https://grid.hpc.cineca.it:9111/CINECA-GALILEO/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core'}

In [11]:
# create client
try:
    site_client = r.site(supercomputer)
except KeyError:
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [10]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers=headers, verify=False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


In [12]:
# check connection to supercomputer
# base = r.site_urls[supercomputer]
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url='https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core', headers=headers, verify=False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPydebian/main/mpich.Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job

This step combines in a single session the following:

- download LFPy container
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation

In [29]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = []

In [30]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    "CPUs": "1",
    "Runtime": "600",
}

In [31]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
sib download --recipe-name lfpydebian_mpich
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [32]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node 
singularity exec lfpydebian_mpich.sif python3 -c "import LFPy; LFPy.run_tests()"
"""

In [33]:
# commands run after job is done
simulation_job["User postcommand"] = "" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [34]:
simulation_job

{'Resources': {'Queue': 'batch', 'CPUs': '1', 'Runtime': '600'},
 'User precommand': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nsib download --recipe-name lfpydebian_mpich\n',
 'RunUserPrecommandOnLoginNode': 'true',
 'Executable': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nunset DISPLAY  # matplotlib may look for a nonexistant display on compute node \nsingularity exec lfpydebian_mpich.sif python3 -c "import LFPy; LFPy.run_tests()"\n',
 'User postcommand': '',
 'RunUserPostcommandOnLoginNode': 'true'}

In [35]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [36]:
# wait while job is running
while job.is_running():
    sleep(10)

In [37]:
job.working_dir.listdir().keys()

dict_keys(['UNICORE_SCRIPT_EXIT_CODE', 'stderr', 'lfpydebian_mpich.sif', 'bss_submit_1603720273767', 'stdout', '.UNICORE_POST_0/', '.UNICORE_PRE_0/', 'UNICORE_Job_1603720273767'])

In [38]:
# STDERR output (if any)
stderr = job.working_dir.stat('stderr')
pprint(stderr.raw().readlines()[:5])
pprint(stderr.raw().readlines()[-5:])

[b'        Preparing the environment for use of requested stage ( 2020 ).\n',
 b'    \n',
 b'................./usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: Us'
 b"erWarning: 0 sections detected! Consider setting 'delete_sections=True'\n",
 b'  warn(mssg)\n',
 b'./usr/local/lib/python3.7/dist-packages/LFPy/cell.py:185: UserWarning: 4 sec'
 b"tions detected! Consider setting 'delete_sections=True'\n"]
[b'..\n',
 b'----------------------------------------------------------------------\n',
 b'Ran 2 tests in 0.548s\n',
 b'\n',
 b'OK\n']


In [40]:
# STDOUT output
stdout = job.working_dir.stat('stdout')
pprint(stdout.raw().readlines())

[b'Mechanisms already loaded from path: /usr/local/lib/python3.7/dist-packages/'
 b'LFPy/test.  Aborting.\n',
 b'Mechanisms already loaded from path: /usr/local/lib/python3.7/dist-packages/'
 b'LFPy/test.  Aborting.\n',
 b'Mechanisms already loaded from path: /usr/local/lib/python3.7/dist-packages/'
 b'LFPy/test.  Aborting.\n',
 b'Mechanisms already loaded from path: /usr/local/lib/python3.7/dist-packages/'
 b'LFPy/test.  Aborting.\n',
 b'\n',
 b'test LFPy.Cell class and methods:\n',
 b'\n',
 b'\n',
 b'\n',
 b'test LFPy.TemplateCell class and methods:\n',
 b'\n',
 b'test LFPy.lfpcalc methods:\n',
 b'\n',
 b'test LFPy.RecExtElectrode class and methods:\n',
 b'\n',
 b'test LFPy.NetworkCell class and methods:\n',
 b'\n',
 b'test LFPy.NetworkPopulation class and methods:\n',
 b'\n',
 b'test LFPy.Network class and methods:\n',
 b'Connected population test to test by 5 connections and 19 synapses\n',
 b't = 100.0 ms\n',
 b'Connected population test to test by 6 connections and 24 synapses\n'

In [41]:
# kill job, clean up files on the remote
job.delete()